# Structured Output with Anthropic
In order to build reliable pipelines in which LLMs consistent return output in the same format, we are using a **Structured Output**
- This means that we define a blueprint for the output
- We pass the 'blueprint' to the LLM
- Then the LLM output will confirm to the blueprint.

This 'blueprint' in the LLM jargon is often called a "schema".

**To generate structured outputs with Anthropic, we'll use the library `instructor`**

In [1]:
!pip install anthropic instructor --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.9 MB/s eta 0:00:00


In [2]:
import anthropic
from google.colab import userdata
from pydantic import BaseModel
from enum import Enum
import instructor

ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

We use pydantic to create data models.

In [3]:
class City(str, Enum):
    aria = "Aria"
    kniga = "Kniga"
    aquabah = "Aquabah"
    torini = "Torini"

class Character(BaseModel):
    name: str
    age: int
    city: City
    job: str
    two_sentences_background_story: str
    inventory: list[str]

In [4]:
antropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
client = instructor.from_anthropic(antropic_client)

# note that client.chat.completions.create will also work
player = client.messages.create(
    model="claude-3-5-haiku-latest",
    max_tokens=1024,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful table-top RPG gamemaster assistant."},
        {
            "role": "user",
            "content": "Generate an Alchemist character for a low-fantasy RPG campaign. Be creative"},
    ],
    temperature=0.9,
    response_model=Character,
)

Now we can see the `player` object which has been created.

In [5]:
player

Character(name='Lyra Morningmist', age=35, city=<City.torini: 'Torini'>, job='Alchemist', two_sentences_background_story='Lyra was once a promising court alchemist who fell from grace after an experimental potion caused an unexpected magical mishap during a royal ceremony. Now she works as an independent researcher, seeking to redeem her reputation by creating groundbreaking elixirs that could change the world of medicine and magic.', inventory=['brass mortar and pestle', 'leather-bound alchemy journal', 'collection of rare herb vials', 'distillation apparatus', 'emergency healing tincture'])

In [6]:
# You can access individual fields
player.name, player.age, player.city.name

('Lyra Morningmist', 35, 'torini')

Learn more about instructor here: https://python.useinstructor.com/